In [ ]:
from pathlib import Path
import pandas as pd

from mth5.clients.make_mth5 import MakeMTH5
from mt_metadata.utils.mttime import get_now_utc, MTime

**Initialize a MakeMTH5 object**: Setting the client to "IRIS".  Here, we are using `obspy.clients` tools for the request.  Here are the available [FDSN clients](https://docs.obspy.org/packages/obspy.clients.fdsn.html). 

In [ ]:
m = MakeMTH5()
m.client = "IRIS"

**Make the data inquiry as a DataFrame**

The DataFrame has columns:

| Column Name         |   Description                                                                                                 |
| ------------------- | --------------------------------------------------------------------------------------------------------------|
| **network**         | [FDSN Network code (2 letters)](http://www.fdsn.org/networks/)                                                |
| **station**         | [FDSN Station code (usually 5 characters)](https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming/)|
| **location**        | [FDSN Location code (typically not used for MT)](http://docs.fdsn.org/projects/source-identifiers/en/v1.0/location-codes.html) |
| **channel**         | [FDSN Channel code (3 characters)](http://docs.fdsn.org/projects/source-identifiers/en/v1.0/channel-codes.html)|
| **start**           | Start time (YYYY-MM-DDThh:mm:ss) UTC |
| **end**             | End time (YYYY-MM-DDThh:mm:ss) UTC  |

In [ ]:
channels = ["LFE", "LFN", "LFZ", "LQE", "LQN"]
CAY10 = ["EM","CAY10", '2019-10-07T00:00:00', '2019-10-30T00:00:00']
CAS04 = ["ZU", "CAS04",  '2020-06-02T19:00:00', '2020-07-13T19:00:00'] 
NVR08 = ["ZU", "NVR08", '2020-06-02T19:00:00', '2020-07-13T19:00:00']

request_list = []
for entry in [CAY10, CAS04, NVR08]:
    for channel in channels:
        request_list.append([entry[0], entry[1], "", channel, entry[2], entry[3]])

# Turn list into dataframe
request_df =  pd.DataFrame(request_list, columns=m.column_names)

**Get only the metadata from IRIS**:  It can be helpful to make sure that your request is what you would expect.  For that you can request only the metadata from IRIS.  The request is quick and light so shouldn't need to worry about the speed.  

In [ ]:
inventory, data = m.get_inventory_from_df(request_df, data=False)

**Make an MTH5 from a request**: we are setting `interact=False` which closes the file HDF5 file when finished. If you want to interrogate the file after its made set `interact=True`. 

In [ ]:
begin = MTime(get_now_utc())

mth5_object = m.make_mth5_from_fdsnclient(request_df, interact=False)
ch_summary = mth5_object.channel_summary
ch_summary.to_csv("channel_summary.csv")

end = MTime(get_now_utc())

print(f"Created {mth5_object.filename} and channel summary 'channel_summary.csv').")
print("Took {(int(end - begin) // 60)}:{(end - begin) % 60:05.2f} minutes")

In [ ]:
begin = MTime(get_now_utc())

ch_summary = mth5_object.channel_summary
ch_summary.to_csv("channel_summary.csv")

end = MTime(get_now_utc())

print(f"Created channel summary 'channel_summary.csv')")
print(f"Took {(int(end - begin) // 60)}:{(end - begin) % 60:05.2f} minutes")

In [ ]:
ch_summary